Appunti:Prendo le def
calcolo i termini più frequenti
faccio disambiguation
ottengo synset piu frequenti - che saranno i possibili genus
confronto
Il punto è non cofrontare tutte le definizioni (100K+) ma confrontarle solo con i genus

In [10]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from nltk.corpus import wordnet as wn
from nltk.stem.snowball import SnowballStemmer

In [28]:
# Extract the data from csv
df = pd.DataFrame(pd.read_csv('../def.csv'))

emotion_defs = df.loc[0,"P1":].dropna().replace(',','', regex=True)

# Create a list of tokenaized definitions and apply morphy function
list_defs = [] # List of single words of definitions
list_defs_m = [] # List of single words of definitions after morphy
for definition in emotion_defs:
    def_tokens = word_tokenize(definition) # ['Range', 'of', 'concepts', 'human', 'beings', 'feel', 'in', 'certain', 'situations']
    list_defs.append(def_tokens)
    for word in def_tokens:
        list_defs_m.append(wn.morphy(word))
    
print(list_defs)
print("After Morphy:")
print(list_defs_m)

[['Range', 'of', 'concepts', 'human', 'beings', 'feel', 'in', 'certain', 'situations'], ['Something', 'you', 'can', 'feel'], ['Something', 'that', 'an', 'animal', 'can', 'feel'], ['something', 'you', 'think', 'that', 'makes', 'you', 'feel', 'good', 'or', 'bad'], ['Human', 'sensation', 'arising', 'from', 'the', 'form', 'of', 'feelings'], ['State', 'of', 'mind', 'that', 'a', 'living', 'being', 'can', 'percieve'], ['Feeling', 'that', 'a', 'human', 'or', 'an', 'animal', 'can', 'express', 'towards', 'others'], ['A', 'sentiment', 'that', 'a', 'living', 'entity', 'can', 'feel', 'and', 'express', 'throw', 'the', 'word', 'and', 'their', 'body', '.'], ['what', 'you', 'feel', 'in', 'a', 'certain', 'moment'], ['feeling', 'experienced', 'by', 'sentient', 'beings', 'and', 'cause', 'by', 'events', 'happening', 'in', 'the', 'outside', 'world', 'as', 'well', 'as', 'in', 'the', 'being', "'s", 'ming'], ['Emotion', 'is', 'a', 'feeling', 'that', 'is', 'expressed', 'through', 'facial', 'expressions', 'body'

In [19]:
syns = wn.synsets("range")
syns2 = wn.synsets("rang")
print(syns)
print(syns2)
print(syns[0].definition())

definition = syns[0].definition()


[Synset('scope.n.01'), Synset('range.n.02'), Synset('range.n.03'), Synset('range.n.04'), Synset('range.n.05'), Synset('range.n.06'), Synset('image.n.07'), Synset('compass.n.03'), Synset('stove.n.01'), Synset('range.v.01'), Synset('roll.v.12'), Synset('range.v.03'), Synset('range.v.04'), Synset('range.v.05'), Synset('crop.v.05'), Synset('range.v.07'), Synset('rate.v.01')]
[Synset('ring.v.01'), Synset('resound.v.01'), Synset('ring.v.03'), Synset('call.v.03'), Synset('surround.v.01'), Synset('ring.v.06')]
an area in which something acts or operates or has power or control:
